In [12]:
import logging
import constants
import boto3
from botocore.exceptions import ClientError
import os

In [13]:
# use third party object storage to create an S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=constants.AWS_S3_ENDPOINT,
    aws_access_key_id=constants.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=constants.AWS_SECRET_ACCESS_KEY,
)

bucket = constants.AWS_S3_BUCKET

ValueError: Invalid endpoint: test

In [3]:
#list contents of bucket 
response = s3_client.list_objects_v2(Bucket=bucket)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        print(obj['Key'])
        print(obj['Size'])
else:
    print("No objects found in the bucket.")

2017_wildfire_imagery/C10683/wvge_c10683_c1r1_2017_bcalb_500mm_fcir_enh.sid
42258526
2017_wildfire_imagery/C10683/wvge_c10683_c1r1_2017_bcalb_500mm_rgb_enh.sid
28932373
STAC_DEM/
0
STAC_DEM/Catalog/DEM_Test.json
1951
STAC_DEM/Catalog/bc_092o018_xli1m_utm10_2018_2_cog/bc_092o018_xli1m_utm10_2018_2_cog.json
1132
STAC_DEM/Catalog/bc_092o018_xli1m_utm10_2018_cog/bc_092o018_xli1m_utm10_2018_cog.json
1126
STAC_DEM/Catalog/bc_092o019_xli1m_utm10_2018_2_cog/bc_092o019_xli1m_utm10_2018_2_cog.json
1132
STAC_DEM/Catalog/bc_092o019_xli1m_utm10_2018_cog/bc_092o019_xli1m_utm10_2018_cog.json
1126
STAC_DEM/Catalog/bc_092o020_xli1m_utm10_2018_2_cog/bc_092o020_xli1m_utm10_2018_2_cog.json
1132
STAC_DEM/Catalog/bc_092o020_xli1m_utm10_2018_cog/bc_092o020_xli1m_utm10_2018_cog.json
1126
STAC_DEM/Catalog/bc_092o028_xli1m_utm10_2018_cog/bc_092o028_xli1m_utm10_2018_cog.json
1126
STAC_DEM/Catalog/bc_092o029_xli1m_utm10_2019_cog/bc_092o029_xli1m_utm10_2019_cog.json
1126
STAC_DEM/Catalog/bc_092o030_xli1m_utm10_201

In [4]:
# specify the folder path (in the current working directory) for your object 
# if the folder doesn't exist, create it
save_path = os.path.join(os.getcwd(), r'junk')
if not os.path.exists(save_path):
    os.makedirs(save_path)

# create new text file within the folder
dest_file_path = os.path.join(save_path, r'test.txt')

# write data to the new text file
with open(dest_file_path, 'w') as fh:
    fh.write('example example example\n')

In [ ]:
# put the new text file into your object storage bucket
s3_client.put_object(Bucket=bucket, Key=dest_file_path)

In [5]:
# define the function for creating presigned URL 
# running the function will return a shareable URL in the print output
def create_presigned_url(
    bucket_name = bucket,
    object_name = dest_file_path,
    expiration=180,
):

    try:
        response = s3_client.generate_presigned_url(
            "get_object",
            Params={"Bucket": bucket_name, "Key": object_name},
            ExpiresIn=expiration,
            )
        print(response)

    except ClientError as e:
        logging.error(e)
        return None

    # The response contains the presigned URL
    return response

In [ ]:
# run the presigned URL function with the given parameters
create_presigned_url(bucket, dest_file_path)

In [4]:
# delete the remote object
s3_client.delete_object(
    Bucket=bucket, 
    Key='DEM_Test.json'
    )

{'ResponseMetadata': {'RequestId': '8e22ee0d:18fb3161e1b:9988:3f67',
  'HostId': 'b04a9f110eba509b97b1110dfa30c33c9a6d8abab09f6ea12b617d45e2c37c27',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Thu, 30 May 2024 14:47:29 GMT',
   'server': 'ViPR/1.0',
   'x-amz-request-id': '8e22ee0d:18fb3161e1b:9988:3f67',
   'x-amz-id-2': 'b04a9f110eba509b97b1110dfa30c33c9a6d8abab09f6ea12b617d45e2c37c27',
   'x-amz-version-id': '1717080449671',
   'x-amz-delete-marker': 'true',
   'x-emc-previous-object-size': '0',
   'content-length': '0'},
  'RetryAttempts': 0},
 'DeleteMarker': True,
 'VersionId': '1717080449671'}

In [1]:
# add folder to S3 bucket 
def uploadDirectory(path,bucketname):
        for root,dirs,files in os.walk(path):
            for file in files:
                s3_client.upload_file(os.path.join(root,file),bucketname,file)

In [8]:
#download jsons
#list contents of bucket 
response = s3_client.list_objects_v2(Bucket=bucket,Prefix='STAC_DEM/Catalog' )

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        create_presigned_url(bucket,obj['Key'])
        
else:
    print("No objects found in the bucket.")


https://nrs.objectstore.gov.bc.ca:443/cloudgistest/STAC_DEM/Catalog/DEM_Test.json?AWSAccessKeyId=nr-cloudgis-test&Signature=2ZEL0qhz4HCMd5JxJNxPUAJdxNc%3D&Expires=1720045329
https://nrs.objectstore.gov.bc.ca:443/cloudgistest/STAC_DEM/Catalog/bc_092o018_xli1m_utm10_2018_2_cog/bc_092o018_xli1m_utm10_2018_2_cog.json?AWSAccessKeyId=nr-cloudgis-test&Signature=3pVgxQNwtDQgZS9pNLIyILdx2nA%3D&Expires=1720045329
https://nrs.objectstore.gov.bc.ca:443/cloudgistest/STAC_DEM/Catalog/bc_092o018_xli1m_utm10_2018_cog/bc_092o018_xli1m_utm10_2018_cog.json?AWSAccessKeyId=nr-cloudgis-test&Signature=GOFEj20k52Va3pesaR4OBFO%2Fc0Y%3D&Expires=1720045329
https://nrs.objectstore.gov.bc.ca:443/cloudgistest/STAC_DEM/Catalog/bc_092o019_xli1m_utm10_2018_2_cog/bc_092o019_xli1m_utm10_2018_2_cog.json?AWSAccessKeyId=nr-cloudgis-test&Signature=zv4M4K0lT3M1%2BWk84D2SiUpOMzE%3D&Expires=1720045329
https://nrs.objectstore.gov.bc.ca:443/cloudgistest/STAC_DEM/Catalog/bc_092o019_xli1m_utm10_2018_cog/bc_092o019_xli1m_utm10_2018

In [9]:
#delete stac jsons
#list contents of bucket 
response = s3_client.list_objects_v2(Bucket=bucket,Prefix='STAC_DEM/Catalog' )

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        print(obj['Key'])
        s3_client.delete_object(
            Bucket=bucket, 
            Key=obj['Key']
            )
else:
    print("No objects found in the bucket.")


No objects found in the bucket.
